In [7]:

import gensim
import pandas as pd

In [1]:
# Exports data from corpus
sentences, cur_sent = list(), list()
with open('./wackypedia_en1.words10.20Mwords') as f:
 for line in f:
    line = line.strip()
    if line == '</s>':
        sentences.append(cur_sent)
        cur_sent = list()
    elif line != '<s>' and not line.startswith('<text') and not line.startswith('</text'):
        cur_sent.append(line.split('\t')[0])

In [12]:
# Builds the models
model_big_ten = gensim.models.Word2Vec(sentences, min_count=5, window=1, vector_size=10)
model_small_ten = gensim.models.Word2Vec(sentences, min_count=5, window=10, vector_size=10)

model_big_five = gensim.models.Word2Vec(sentences, min_count=5, window=1, vector_size=500)
model_small_five = gensim.models.Word2Vec(sentences, min_count=5, window=10, vector_size=500)

models = [model_big_ten, model_big_five, model_small_ten, model_small_five]

In [ ]:

# Makes the similarity lists
words = pd.read_csv('./SimLex-999/SimLex-999.txt', delimiter='\t')
pairs = list(zip(words['word1'],words['word2']))
results = [[], [], [], []]


for pair in pairs:
    for index, model in enumerate(models):
        try:
            similarity = models[index].wv.similarity(pair[0], pair[1])
        except KeyError:
            similarity = 0
        finally:
            results[index].append(similarity)
